In [1]:
import numpy as np
import pandas as pd
from math import exp
import copy
from sklearn import preprocessing

In [32]:
# Feed Forward helper methods
def sigmoid(value):
    return 1.0/(1+exp(value * (-1)))

def sigma(matrix_weight, matrix_input, bias=0):
    # Prereq: len(arr_weight) = len(arr_input)
    return matrix_weight.dot(matrix_input.transpose()) + bias


# hidden_layer = int (number of hidden layers)
# nb_nodes = arr[int] (number of nodes per hidden layer)
# len_input_matrix = int (number of features)
# Output: List of Matrixes
# Method: He initialization
# Link: https://towardsdatascience.com/weight-initialization-techniques-in-neural-networks-26c649eb3b78
def initialize_weights(hidden_layer, nb_nodes, len_input_matrix):
    nb_nodes = nb_nodes.astype(int) #diperlukan karena dianggap float dalam fungsi randn jika tak diubah
    arr_weight_this_batch = list()
    for i in range(hidden_layer):
        if i==0:
            nb_nodes_prev = len_input_matrix
        else:
            nb_nodes_prev = nb_nodes[i-1]
        weight_matrix = np.random.randn(nb_nodes[i], nb_nodes_prev) * np.sqrt(2/(nb_nodes_prev+nb_nodes[i]))
        arr_weight_this_batch.append(weight_matrix)
    
    return arr_weight_this_batch

In [33]:
# Backpropagation and Update Weight helper methods
def error(feed_forward_output, target_output):
    return 0.5*((target_output-feed_forward_output)**2)

def propagate_error_output_layer(feed_forward_output, target_output):
    return feed_forward_output*(1-feed_forward_output)*(target_output-feed_forward_output)

def propagate_error_hidden_layer_neuron(arr_weight_input, arr_neuron_input, arr_weight_output, arr_neuron_output):
    #Input here means input in a single neuron, while output means output of a single neuron
    #len(arr_weight) = len(arr_input)
    sigma_input, sigma_output = sigma(arr_weight_input, arr_neuron_input), sigma(arr_weight_output, arr_neuron_output)
    return sigmoid(sigma_input) * (1 - sigmoid(sigma_input)) * sigma_output

# error = neuron's error
def update_weight_neuron(weight_prev_prev, weight_prev, learning_rate, momentum, error, input_neuron):
    # weight_prev_prev = previous of weight_prev
    return weight_prev + weight_prev_prev * momentum + learning_rate*error*input_neuron

In [34]:
# input_matrix = matrix[float] (data) (asumsi, kolom terakhir adalah hasil klasifikasi)
# hidden_layers = int (number of hidden layers)
# nb_nodes = arr[int] (number of nodes per hidden layer)
# nu = float (momentum)
# alfa = float (learning rate)
# epoch = int (number of training loops)
# batch_size = int (mini-batch)
# output = FFNN prediction model (list of matrix)
def mini_batch_gradient_descent(input_matrix, hidden_layer, nb_nodes, nu, alfa, epoch, batch_size=1):
    
    #transpose-slicing, memisah input dan label
    col_width = input_matrix.shape[1]
    input_data = (input_matrix.transpose()[0:col_width-1]).transpose()
    label_data = (input_matrix.transpose()[col_width-1:col_width]).transpose()
    #print(input_data, "\n")
    #print(label_data, "\n")
    
    hidden_layer += 1
    nb_nodes = np.append(nb_nodes, [1])
    for no_epoch in range(epoch):
        if no_epoch == 0:
            arr_weight_this_batch = initialize_weights(hidden_layer, nb_nodes, col_width-1)
        else:
            arr_weight_prev_batch = copy.deepcopy(arr_weight_this_batch) # tracking previous state of weights
            
        for no_input_data in range(len(input_data)):
            #Feed Forward
            all_sigma_values = list()
            for no_hidden_layer in range(hidden_layer):
                if no_hidden_layer == 0:
                    all_sigma_values.append(sigma(arr_weight_this_batch[no_hidden_layer], input_data[no_input_data]))
                else:
                    all_sigma_values.append(sigma(arr_weight_this_batch[no_hidden_layer], all_sigma_values[no_hidden_layer-1]))
                for no_rows in range(len(all_sigma_values[no_hidden_layer])):
                    all_sigma_values[no_hidden_layer][no_rows] = sigmoid(all_sigma_values[no_hidden_layer][no_rows])
            #Result of sigma will be array with 1 element only, so it's safe to select like this
            error_value = error(all_sigma_values[no_hidden_layer][0], label_data[no_input_data])[0]
            print("From data in epoch " + str(no_epoch) + " and no_input " + str(no_input_data) + " has error " + str(error_value))
            
            #Back Propagation
            output_error = propagate_error_output_layer(all_sigma_values[no_hidden_layer][0], label_data[no_input_data])
            #for no_hidden_layer in range(hidden_layer-1, -1, -1):
                #print(no_hidden_layer)
            #print(hidden_layer)
            #update_weight_neuron(weight_prev_prev, weight_prev, nu, alfa, error, output_neuron):
    #return output

In [35]:
# dataset load
#df = pd.read_csv("weather.csv")
csv_string = input("Input .csv filename: ")
try:
    df = pd.read_csv(csv_string)
except:
    print("File not found.")
    ###quit() (di file Python, pake ini)
print("File loaded successfuly.")
#print(df.head, "\n")

Input .csv filename: weather.csv
File loaded successfuly.


In [36]:
 # dataset preprocess

# transform non-numeric data to numeric data

types = df.dtypes
labels = df.columns.values # because pandas select columns using column names
def transform_to_numeric(matrix_data):
    for i in range(matrix_data.shape[1]):
        type_i = types[i]
        if (type_i == object):
            values = matrix_data[labels[i]].unique()
            dict_i = dict(zip(values, range(len(values)))) # transform every unique object/string into numbers
            matrix_data = matrix_data.replace({labels[i]:dict_i})
        elif (type_i == bool):
            matrix_data[labels[i]] = matrix_data[labels[i]].astype(int)
    return matrix_data

newdf = transform_to_numeric(df)
#print(newdf.head, "\n")

# scaling
def scale_data(matrix_data, min_val, max_val):
    
    def scaling(value):
        return (value - minValue)*(max_val - min_val)/(maxValue - minValue) + min_val
    
    for x in range(matrix_data.shape[1]):
        minValue = matrix_data[labels[x]].min()
        maxValue = matrix_data[labels[x]].max()
        matrix_data[labels[x]] = matrix_data[labels[x]].apply(scaling)
    return matrix_data

data_matrix = scale_data(newdf, 0, 1)
#print(data_matrix, "\n")
data_matrix = data_matrix.to_numpy() #convert pandas dataframe to numpy array
#print(data_matrix, "\n")

In [37]:
# input and main program

while True:
    hidden_layers = int(input("Input number of hidden layers: "))
    if (hidden_layers <= 10 and hidden_layers >= 0):
        break
    else:
        print("# of hidden layers must be a positive integer and no more than 10.")

nb_nodes = np.empty(hidden_layers)
for i in range(hidden_layers):
    while True:
        nb_nodes[i] = int(input("Input number of nodes for hidden layer %d : " % i))
        if (nb_nodes[i] > 0):
            break
        else:
            print("# of nodes must be a positive integer.")
    
    
while True:
    momentum = float(input("Input momentum: "))
    if (momentum <= 1 and momentum >= 0):
        break
    else:
        print("Momentum must be between 0 and 1.")

while True:
    learning_rate = float(input("Input learning rate: "))
    if (learning_rate <= 1 and learning_rate >= 0):
        break
    else:
        print("Learning rate must be between 0 and 1.")

while True:
    epoch = int(input("Input epoch: "))
    if (epoch > 0):
        break
    else:
        print("Epoch must be a positive integer.")

while True:
    batch_size = int(input("Input the batch size: "))
    if (batch_size > 0):
        break
    else:
        print("Batch size must be a positive integer.")

result = mini_batch_gradient_descent(data_matrix, hidden_layers, nb_nodes, momentum, learning_rate, epoch, batch_size)

Input number of hidden layers: 3
Input number of nodes for hidden layer 0 : 2
Input number of nodes for hidden layer 1 : 3
Input number of nodes for hidden layer 2 : 4
Input momentum: 0.1
Input learning rate: 0.1
Input epoch: 1
Input the batch size: 1
From data in epoch 0 and no_input 0 has error 0.08273198438309974
From data in epoch 0 and no_input 1 has error 0.08259375907484069
From data in epoch 0 and no_input 2 has error 0.17596725090045667
From data in epoch 0 and no_input 3 has error 0.17603134132571704
From data in epoch 0 and no_input 4 has error 0.17614367986585916
From data in epoch 0 and no_input 5 has error 0.08241020045169105
From data in epoch 0 and no_input 6 has error 0.17647668912485645
From data in epoch 0 and no_input 7 has error 0.0826845000308655
From data in epoch 0 and no_input 8 has error 0.17620566186326722
From data in epoch 0 and no_input 9 has error 0.17607231533807688
From data in epoch 0 and no_input 10 has error 0.17633455460254488
From data in epoch 0 a